## Topic Modeling - Wissenschaftler*innen

In [ ]:
!pip install bertopic

In [3]:
from bertopic import BERTopic

import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Daten einlesen & Preprocessing

In [4]:
df1 = pd.read_excel('tweets_wissenschaftlerinnen.xlsx')
df2 = pd.read_excel('tweets_wissenschaftler.xlsx')

In [5]:
df1['tweet_text'] = df1['tweet_text'].astype(str)
df1['retweet_text_retweeted_tweet'] = df1['retweet_text_retweeted_tweet'].astype(str)
df1['zitat_text'] = df1['zitat_text'].astype(str)
df2['tweet_text'] = df2['tweet_text'].astype(str)
df2['retweet_text_retweeted_tweet'] = df2['retweet_text_retweeted_tweet'].astype(str)
df2['zitat_text'] = df2['zitat_text'].astype(str)

In [6]:
def classify_tweet(row):
    if row['tweet_text'] != '-998' and row['retweet_text_retweeted_tweet'] == '-998' and row['zitat_text'] == '-998':
        return 'tweet'
    elif row['retweet_text_retweeted_tweet'] != '-998':
        return 'retweet'
    elif row['tweet_text'] != '-998' and row['zitat_text'] != '-998':
        return 'zitat_tweet'
    return 'unclassified'

In [7]:
df1['tweet_class'] = df1.apply(classify_tweet, axis=1)
df2['tweet_class'] = df2.apply(classify_tweet, axis=1)

In [8]:
def gather_texts(row):
    texts = []
    for column in ['tweet_text', 'retweet_text_retweeted_tweet', 'zitat_text']:
      if pd.notna(row[column]) and str(row[column]) != '-998':
            texts.append(row[column])
    return ' '.join(texts)

In [9]:
df1['all_tweets'] = df1.apply(gather_texts, axis=1)
df2['all_tweets'] = df2.apply(gather_texts, axis=1)

In [10]:
def preprocess_tweet(text):
    text = re.sub(r'@\w+', '', text)  # Entfernt Benutzernamen
    text = re.sub(r'http\S+', '', text)  # Entfernt URLs
    text = re.sub(r'www.\S+', '', text)  # Entfernt URLs
    text = re.sub(r'[0-9]+', '', text)  # Entfernt Zahlen
    text = re.sub(r'[^\w\s]', '', text)  # Entfernt Sonderzeichen
    text = re.sub(r'_', '', text)  # Entfernt Unterstriche
    text = re.sub(r'\s+', ' ', text).strip()  # Ersetzt mehrere Leerzeichen durch ein einzelnes und entfernt sie am Anfang/Ende
    text = re.sub(r'-998', '', text)  # entfernt fehlende Werte
    text = re.sub(r'nan', '', text)  # entfernt fehlende Werte

    return text

In [11]:
df1['all_tweets'] = df1['all_tweets'].apply(preprocess_tweet)
df2['all_tweets'] = df2['all_tweets'].apply(preprocess_tweet)

In [12]:
df = pd.concat([df1, df2], ignore_index=True)

### Topic Modeling mit BERTopic

In [14]:
docs1 = list(df1.all_tweets.values)
docs2 = list(df2.all_tweets.values)
docs = list(df.all_tweets.values)

german_stopwords = set(stopwords.words('german'))
stopwords = list(german_stopwords)
vectorizer_model = CountVectorizer(stop_words=stopwords)

## Wissenschaftlerinnen

#### Erstes Modell ohne Beschränkung der Themenanzahl




In [14]:
# BERTopic-Modell erstellen
model1 = BERTopic(language="German", calculate_probabilities=True, verbose=True, vectorizer_model=vectorizer_model)

# Model anwenden und Topics extrahieren
topics, probs = model1.fit_transform(docs1)

2024-04-27 08:51:07,668 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

2024-04-27 08:52:44,236 - BERTopic - Embedding - Completed ✓
2024-04-27 08:52:44,242 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-27 08:53:00,126 - BERTopic - Dimensionality - Completed ✓
2024-04-27 08:53:00,132 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-27 08:53:00,272 - BERTopic - Cluster - Completed ✓
2024-04-27 08:53:00,287 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-27 08:53:00,444 - BERTopic - Representation - Completed ✓


In [15]:
# Ergebnisse
print(f"Anzahl der Topics: {model1.get_topic_info().shape[0] - 1}")  # Minus 1 wegen des Outliers-Topics (-1)
topic_info1 = model1.get_topic_info()
topic_info1.head(20)

Anzahl der Topics: 29


,Topic,Count,Name,Representation,Representative_Docs
0,-1,424,-1_forschung_mehr_wer_geht,"[forschung, mehr, wer, geht, phdlife, neue, bi...",[Matéria na Portadores de esperança para Dez p...
1,0,129,0_forschung_ichbinhanna_stellen_drittmittel,"[forschung, ichbinhanna, stellen, drittmittel,...",[Existenzangst killt innovative Forschung Wer ...
2,1,90,1_frauen_genderstudies_personen_gender,"[frauen, genderstudies, personen, gender, fors...",[Wenn du über Jahre ach was Jahrzehnte in nerv...
3,2,49,2_forschung_politik_politiksprüche_zib,"[forschung, politik, politiksprüche, zib, verb...",[Immer wieder dieselben Politiksprüche beim Kl...
4,3,37,3_behaviour_verhalten_planetary_health,"[behaviour, verhalten, planetary, health, inst...",[Der Klimawandel erfordert jetzt weitreichende...
5,4,31,4_fdm_forschungsdatenmanagement_reputation_data,"[fdm, forschungsdatenmanagement, reputation, d...",[FDM ist wichtig va über die Drittmitteleinwer...
6,5,31,5_deutschland_transfer_empfehlungen_schnelleren,"[deutschland, transfer, empfehlungen, schnelle...",[Schwatzen ist in bei der Regiering und ihren ...
7,6,30,6_genchatde_fähigkeiten_drittmittelanträge_bei...,"[genchatde, fähigkeiten, drittmittelanträge, b...",[Erfolgreiche Drittmittelanträge schreiben kön...
8,7,26,7_geht_gebraucht_china_engagiert,"[geht, gebraucht, china, engagiert, ansinnen, ...",[Manche Menschen verstehen einfach nicht was d...
9,8,25,8_wegen_ichbinhanna_wissenschaft_arbeitsbeding...,"[wegen, ichbinhanna, wissenschaft, arbeitsbedi...",[Ihr kennt diesen Spruch Menschen kommen wegen...


In [16]:
model1.visualize_topics()

#### Reduzierung der Themenanzahl

In [17]:
# noch verschiedene Anzahlen testen
model1.reduce_topics(docs1, nr_topics=20)

2024-04-27 08:53:51,090 - BERTopic - Topic reduction - Reducing number of topics
2024-04-27 08:53:51,217 - BERTopic - Topic reduction - Reduced number of topics from 30 to 20


In [18]:
# Ergebnisse
print(f"Anzahl der Topics: {model1.get_topic_info().shape[0] - 1}")  # Minus 1 wegen des Outliers-Topics (-1)
topic_info_reduced1 = model1.get_topic_info()
topic_info_reduced1

Anzahl der Topics: 19


,Topic,Count,Name,Representation,Representative_Docs
0,-1,424,-1_forschung_mehr_wer_geht,"[forschung, mehr, wer, geht, phdlife, neue, bi...",[Matéria na Portadores de esperança para Dez p...
1,0,221,0_forschung_ichbinhanna_lehre_mehr,"[forschung, ichbinhanna, lehre, mehr, immer, g...",[Existenzangst killt innovative Forschung Wer ...
2,1,90,1_frauen_genderstudies_forschung_personen,"[frauen, genderstudies, forschung, personen, g...",[Wenn du über Jahre ach was Jahrzehnte in nerv...
3,2,63,2_deutschland_unbefristet_forschung_team,"[deutschland, unbefristet, forschung, team, be...",[Mitarbeiterstelle noch eine im Bereich Politi...
4,3,63,3_klima_geht_gebraucht_china,"[klima, geht, gebraucht, china, engagiert, dem...",[Manche Menschen verstehen einfach nicht was d...
5,4,51,4_freuen_bewerbungen_februar_forschung,"[freuen, bewerbungen, februar, forschung, bewi...",[Work with us Wir freuen uns über Bewerbungen ...
6,5,41,5_ichbinhanna_wegen_befristete_ichbinreyhan,"[ichbinhanna, wegen, befristete, ichbinreyhan,...",[Befristete Verträge und wenig Zeit für Forsch...
7,6,31,6_fdm_forschungsdatenmanagement_data_reputation,"[fdm, forschungsdatenmanagement, data, reputat...",[FDM ist wichtig va über die Drittmitteleinwer...
8,7,31,7_mecfs_jahrzehnten_med_versorgung,"[mecfs, jahrzehnten, med, versorgung, mecfsimb...",[MECFSimBundestag Heute nach Jahrzehnten ohne ...
9,8,24,8_and_the_phdlife_of,"[and, the, phdlife, of, consumption, advertisi...",[NewBook on consumption and advertising to be ...


In [21]:
def get_details(topic_info):
  for i in range(1, 11):
    representation = topic_info.loc[i, 'Representation']
    representative_docs = topic_info.loc[i, 'Representative_Docs']
    print(f"Thema {i}:")
    print(f"Representation: {representation}")
    print(f"Representative_Docs: {representative_docs}")
    print()

In [20]:
get_details(topic_info_reduced1)

Thema 1:
Representation: ['forschung', 'ichbinhanna', 'lehre', 'mehr', 'immer', 'gute', 'stellen', 'ermöglicht', 'gibt', 'drittmittel']
Representative_Docs: ['Existenzangst killt innovative Forschung Wer strategisch forschen muss um weiterbeschäftigt zu werden setzt in Themenwahl Vorgehen Methodik usw auf Sicherheit Scheitern ist Karriererisiko bzw längst bestehende Trends dafür gibts Stellen und Drittmittel IchBinHanna', 'Existenzangst killt innovative Forschung Wer strategisch forschen muss um weiterbeschäftigt zu werden setzt in Themenwahl Vorgehen Methodik usw auf Sicherheit Scheitern ist Karriererisiko bzw längst bestehende Trends dafür gibts Stellen und Drittmittel IchBinHanna', 'Existenzangst killt innovative Forschung Wer strategisch forschen muss um weiterbeschäftigt zu werden setzt in Themenwahl Vorgehen Methodik usw auf Sicherheit Scheitern ist Karriererisiko bzw längst bestehende Trends dafür gibts Stellen und Drittmittel IchBinHanna']

Thema 2:
Representation: ['frauen', '

In [21]:
model1.visualize_topics()

In [22]:
model1.visualize_barchart(top_n_topics = 10, n_words = 5)

In [24]:
model1.visualize_hierarchy()

#### Themen nach Tweet-Kategorie

In [ ]:
topics_per_class1 = model1.topics_per_class(docs1,
    classes=df1.tweet_class)

In [28]:
model1.visualize_topics_per_class(topics_per_class1,
    top_n_topics=10, normalize_frequency = True)

## Wissenschaftler

Erstes Modell ohne Beschränkung der Themenanzahl

In [37]:
# BERTopic-Modell erstellen
model2 = BERTopic(language="German", calculate_probabilities=True, verbose=True, vectorizer_model=vectorizer_model)

# Model anwenden und Topics extrahieren
topics, probs = model2.fit_transform(docs2)

2024-04-27 09:18:19,539 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/56 [00:00<?, ?it/s]

2024-04-27 09:20:14,941 - BERTopic - Embedding - Completed ✓
2024-04-27 09:20:14,951 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-27 09:20:23,577 - BERTopic - Dimensionality - Completed ✓
2024-04-27 09:20:23,581 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-27 09:20:23,823 - BERTopic - Cluster - Completed ✓
2024-04-27 09:20:23,832 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-27 09:20:24,039 - BERTopic - Representation - Completed ✓


In [38]:
# Ergebnisse
print(f"Anzahl der Topics: {model2.get_topic_info().shape[0] - 1}")  # Minus 1 wegen des Outliers-Topics (-1)
topic_info2 = model2.get_topic_info()
topic_info2.head(20)

Anzahl der Topics: 39


,Topic,Count,Name,Representation,Representative_Docs
0,-1,662,-1_forschung_wer_bildung_ganz,"[forschung, wer, bildung, ganz, wissenschaft, ...",[Matéria na Portadores de esperança para Dez p...
1,0,125,0_forschung_ja_schon_lehre,"[forschung, ja, schon, lehre, einfach, ende, g...",[Dreiviertel aller Ideen für ForschungPublikat...
2,1,72,1_masken_corona_covid_heißt,"[masken, corona, covid, heißt, longcovid, pete...",[Endemie heißt nicht dass ein Erreger harmlos ...
3,2,68,2_deutschland_österreich_deutschlands_prozent,"[deutschland, österreich, deutschlands, prozen...",[Wie können wir Wissenschaft und Forschung in ...
4,3,59,3_klimawandel_zib_politiksprüche_verbote,"[klimawandel, zib, politiksprüche, verbote, ve...",[Immer wieder dieselben Politiksprüche beim Kl...
5,4,49,4_mecfs_versorgung_longcovid_mecfsimbundestag,"[mecfs, versorgung, longcovid, mecfsimbundesta...",[MECFSimBundestag Heute nach Jahrzehnten ohne ...
6,5,47,5_zukunftsstrategie_innovation_chancen_technol...,"[zukunftsstrategie, innovation, chancen, techn...",[Von Neuen Züchtungstechniken bis zur Kernfusi...
7,6,46,6_dgrn_forschung_gute_empfehlenswerter,"[dgrn, forschung, gute, empfehlenswerter, rund...",[Der Forschungsbericht der ist soeben erschien...
8,7,38,7_lothar_wieler_dankewieler_rki,"[lothar, wieler, dankewieler, rki, amt, widmen...",[Professor Lothar H Wieler verlässt das RKI Er...
9,8,31,8_duz_europaweit_special_spitze,"[duz, europaweit, special, spitze, innovativer...",[Exzellente Forschung I Innovativer Transfer I...


In [39]:
model2.visualize_topics()

Reduzierung der Themenanzahl

In [40]:
model2.reduce_topics(docs2, nr_topics=20)

2024-04-27 09:20:25,577 - BERTopic - Topic reduction - Reducing number of topics
2024-04-27 09:20:25,962 - BERTopic - Topic reduction - Reduced number of topics from 40 to 20


In [41]:
# Ergebnisse
print(f"Anzahl der Topics: {model2.get_topic_info().shape[0] - 1}")  # Minus 1 wegen des Outliers-Topics (-1)
topic_info_reduced2 = model2.get_topic_info()
topic_info_reduced2

Anzahl der Topics: 19


,Topic,Count,Name,Representation,Representative_Docs
0,-1,662,-1_forschung_wer_wissenschaft_mehr,"[forschung, wer, wissenschaft, mehr, bildung, ...",[Matéria na Portadores de esperança para Dez p...
1,0,303,0_forschung_ja_gibt_innovation,"[forschung, ja, gibt, innovation, schon, wisse...",[Wichtige Tweets die zeigen warum guten klinis...
2,1,144,1_berlin_deutschland_forschung_beschäftigen,"[berlin, deutschland, forschung, beschäftigen,...",[Wusstet ihr dass Berlin DER Heimatort von Wis...
3,2,121,2_mecfs_longcovid_versorgung_mehr,"[mecfs, longcovid, versorgung, mehr, covid, fo...",[Wir planen eine BMG Hotline LongCovid einzufü...
4,3,93,3_lehre_digitale_podcast_thema,"[lehre, digitale, podcast, thema, forschung, d...",[Auch in dieser Woche bieten wir Ihnen wieder ...
5,4,83,4_kernfusion_alternative_cdu_pv,"[kernfusion, alternative, cdu, pv, usa, wasser...",[Und was ist die Alternative in der Akutheit d...
6,5,83,5_klima_geht_engagiert_demonstrieren,"[klima, geht, engagiert, demonstrieren, ehren,...",[Manche Menschen verstehen einfach nicht was d...
7,6,38,6_lothar_wieler_dankewieler_rki,"[lothar, wieler, dankewieler, rki, amt, widmen...",[Professor Lothar H Wieler verlässt das RKI Er...
8,7,31,7_pfizer_gofforschung_pfizers_malone,"[pfizer, gofforschung, pfizers, malone, holpri...",[Sonntagskrimi Legen Sie ihn weg das hier ist ...
9,8,30,8_einstein_stiftung_globalhealth_gesundheit,"[einstein, stiftung, globalhealth, gesundheit,...",[Die Einstein Stiftung begrüßt herzlich als Ei...


In [48]:
get_details(topic_info_reduced2)

Thema 1:
Representation: ['forschung', 'ja', 'gibt', 'innovation', 'schon', 'wissenschaft', 'mehr', 'gute', 'lehre', 'immer']
Representative_Docs: ['Wichtige Tweets die zeigen warum guten klinische Studien ja am besten mehrere mit unterschiedlichem Design so unglaublich wichtig sind Nur weil etwas logisch klingt oder im Labor funktioniert muss es nicht auch im realen Leben funktionieren Ich schreibe jetzt hier in loser Folge immer wenn ich drüber stolpere Dinge auf die in der theoretischen Forschung oder unter Laborbedingungen funktionieren und für die es einen plausiblen Wirkmechanismus gibt die aber in randomisierten Studien versagt haben', 'Wichtige Tweets die zeigen warum guten klinische Studien ja am besten mehrere mit unterschiedlichem Design so unglaublich wichtig sind Nur weil etwas logisch klingt oder im Labor funktioniert muss es nicht auch im realen Leben funktionieren Ich schreibe jetzt hier in loser Folge immer wenn ich drüber stolpere Dinge auf die in der theoretischen Fo

In [43]:
model2.visualize_topics()

In [44]:
model2.visualize_barchart(top_n_topics = 10, n_words = 5)

In [45]:
model2.visualize_heatmap()

Themen nach Tweet-Kategorie

In [46]:
topics_per_class2 = model2.topics_per_class(docs2, classes=df2.tweet_class)

3it [00:00, 20.17it/s]


In [47]:
model2.visualize_topics_per_class(topics_per_class2, top_n_topics=10, normalize_frequency = True)

## Vergleich

In [15]:
# BERTopic-Modell erstellen
model = BERTopic(language="German", calculate_probabilities=True, verbose=True, vectorizer_model=vectorizer_model)

# Model anwenden und Topics extrahieren
topics, probs = model.fit_transform(docs)

2024-04-27 12:40:02,990 - BERTopic - Embedding - Transforming documents to embeddings.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/93 [00:00<?, ?it/s]

2024-04-27 12:43:13,098 - BERTopic - Embedding - Completed ✓
2024-04-27 12:43:13,101 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-27 12:43:38,938 - BERTopic - Dimensionality - Completed ✓
2024-04-27 12:43:38,946 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-27 12:43:40,641 - BERTopic - Cluster - Completed ✓
2024-04-27 12:43:40,663 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-27 12:43:41,184 - BERTopic - Representation - Completed ✓


In [16]:
# Ergebnisse
print(f"Anzahl der Topics: {model.get_topic_info().shape[0] - 1}")  # Minus 1 wegen des Outliers-Topics (-1)
topic_info = model.get_topic_info()
topic_info.head(20)

Anzahl der Topics: 74


,Topic,Count,Name,Representation,Representative_Docs
0,-1,829,-1_forschung_heute_gibt_mehr,"[forschung, heute, gibt, mehr, immer, ja, the,...",[Die CovidForschung macht gerade riesige Forts...
1,0,264,0_forschung_ja_gibt_lehre,"[forschung, ja, gibt, lehre, mehr, immer, frag...",[Die wichtigsten Punkte aus dem Vortrag v Es i...
2,1,119,1_frauen_gender_genderstudies_käthe,"[frauen, gender, genderstudies, käthe, mädchen...",[Um wisskomm in den Gender Studies mit Blick a...
3,2,107,2_deutschland_deutsche_beschlossen_schnelleren,"[deutschland, deutsche, beschlossen, schneller...",[Was richtig innovativ wäre solide Grundfizier...
4,3,84,3_mecfs_mecfsimbundestag_versorgung_aufklärung,"[mecfs, mecfsimbundestag, versorgung, aufkläru...",[Heute stille Visualisierung vor dem Bundestag...
5,4,61,4_desamfornet_prof_keppler_sciencetalks,"[desamfornet, prof, keppler, sciencetalks, kün...",[Es geht weiter mit den ScienceTalks Am diskut...
6,5,47,5_politiksprüche_verbote_zib_verzweifeln,"[politiksprüche, verbote, zib, verzweifeln, se...",[Immer wieder dieselben Politiksprüche beim Kl...
7,6,45,6_wieler_dankewieler_lothar_rki,"[wieler, dankewieler, lothar, rki, amt, widmen...",[Professor Lothar H Wieler verlässt das RKI Er...
8,7,44,7_tiere_tierversuche_artenkenntnis_eingesetzt,"[tiere, tierversuche, artenkenntnis, eingesetz...",[Im Weinbau werden besonders viele Pflanzensch...
9,8,43,8_archäologie_germanen_meldungen_kultur,"[archäologie, germanen, meldungen, kultur, nac...",[Waren die Germanen nur raubeinige Barbaren Di...


In [17]:
model.visualize_topics()

Reduzierung der Themenanzahl

In [18]:
model.reduce_topics(docs, nr_topics=30)

2024-04-27 12:43:44,053 - BERTopic - Topic reduction - Reducing number of topics
2024-04-27 12:43:44,364 - BERTopic - Topic reduction - Reduced number of topics from 75 to 30


In [19]:
# Ergebnisse
print(f"Anzahl der Topics: {model.get_topic_info().shape[0] - 1}")  # Minus 1 wegen des Outliers-Topics (-1)
topic_info_reduced = model.get_topic_info()
topic_info_reduced

Anzahl der Topics: 29


,Topic,Count,Name,Representation,Representative_Docs
0,-1,829,-1_forschung_mehr_gibt_heute,"[forschung, mehr, gibt, heute, immer, ja, the,...",[Wann ist man mit dem Archiv als News schon ma...
1,0,567,0_forschung_ichbinhanna_wissenschaft_mehr,"[forschung, ichbinhanna, wissenschaft, mehr, l...",[Immer wieder dieselben Politiksprüche beim Kl...
2,1,282,1_lehre_forschung_freuen_chatgpt,"[lehre, forschung, freuen, chatgpt, prof, tran...",[Auch in dieser Woche bieten wir Ihnen wieder ...
3,2,227,2_berlin_deutschland_forschung_transfer,"[berlin, deutschland, forschung, transfer, inn...",[Wusstet ihr dass Berlin DER Heimatort von Wis...
4,3,119,3_frauen_gender_genderstudies_käthe,"[frauen, gender, genderstudies, käthe, persone...",[Um wisskomm in den Gender Studies mit Blick a...
5,4,112,4_mecfs_mecfsimbundestag_versorgung_longcovid,"[mecfs, mecfsimbundestag, versorgung, longcovi...",[MECFSimBundestag Heute nach Jahrzehnten ohne ...
6,5,100,5_kernfusion_alternative_energie_cdu,"[kernfusion, alternative, energie, cdu, wasser...",[Und was ist die Alternative in der Akutheit d...
7,6,86,6_heißt_harmlos_therapeutika_endemie,"[heißt, harmlos, therapeutika, endemie, endemi...",[Endemie heißt nicht dass ein Erreger harmlos ...
8,7,85,7_geht_klima_engagiert_klimaschützer,"[geht, klima, engagiert, klimaschützer, ehren,...",[Manche Menschen verstehen einfach nicht was d...
9,8,73,8_archäologie_kultur_germanen_meldungen,"[archäologie, kultur, germanen, meldungen, neu...",[Moin Ich bin Toke wissenschaftlicher Mitarbei...


In [22]:
get_details(topic_info_reduced)

Thema 1:
Representation: ['forschung', 'ichbinhanna', 'wissenschaft', 'mehr', 'lehre', 'immer', 'ja', 'gibt', 'ermöglicht', 'schon']
Representative_Docs: ['Immer wieder dieselben Politiksprüche beim Klima Auf Forschung setzen nicht auf Verbote Es ist zum Verzweifeln Ja Mehr Forschung ist toll Aber gleichzeitig müssen wir JETZT umsetzen was die Forschung der letzten Jahrzehnte uns heute ermöglicht Und das ist viel zib', 'Immer wieder dieselben Politiksprüche beim Klima Auf Forschung setzen nicht auf Verbote Es ist zum Verzweifeln Ja Mehr Forschung ist toll Aber gleichzeitig müssen wir JETZT umsetzen was die Forschung der letzten Jahrzehnte uns heute ermöglicht Und das ist viel zib', 'Immer wieder dieselben Politiksprüche beim Klima Auf Forschung setzen nicht auf Verbote Es ist zum Verzweifeln Ja Mehr Forschung ist toll Aber gleichzeitig müssen wir JETZT umsetzen was die Forschung der letzten Jahrzehnte uns heute ermöglicht Und das ist viel zib']

Thema 2:
Representation: ['lehre', 'fors

#### Themen nach Gender

In [23]:
topics_per_class = model.topics_per_class(docs,
    classes=df.gender)

2it [00:00, 11.51it/s]


In [28]:
topics_per_class

,Topic,Words,Frequency,Class,Name
0,-1,"forschung, gibt, heute, immer, mehr",499,männlich,-1_forschung_mehr_gibt_heute
1,0,"forschung, wissenschaft, ja, mehr, ichbinhanna",352,männlich,0_forschung_ichbinhanna_wissenschaft_meh...
2,1,"lehre, forschung, prof, digitale, freuen",160,männlich,1_lehre_forschung_freuen_chatgpt
3,2,"berlin, deutschland, forschung, transfer, exze...",173,männlich,2_berlin_deutschland_forschung_transfer
4,3,"frauen, käthe, gender, wissenschaft, mädchen",28,männlich,3_frauen_gender_genderstudies_käthe
5,4,"mecfs, versorgung, mecfsimbundestag, longcovid...",55,männlich,4_mecfs_mecfsimbundestag_versorgung_long...
6,5,"kernfusion, alternative, cdu, akutheit, energi...",81,männlich,5_kernfusion_alternative_energie_cdu
7,6,"heißt, viren, harmlos, endemie, therapeutika",53,männlich,6_heißt_harmlos_therapeutika_endemie
8,7,"geht, engagiert, klima, klimaschützer, fleissig",40,männlich,7_geht_klima_engagiert_klimaschützer
9,8,"archäologie, germanen, meldungen, kultur, nach...",45,männlich,8_archäologie_kultur_germanen_meldungen


In [30]:
model.visualize_topics_per_class(topics_per_class,
    top_n_topics=10, normalize_frequency = True)

In [48]:
pd.set_option('display.width', 1000)

In [54]:
topics_per_class[1:7]

,Topic,Words,Frequency,Class,Name
1,0,"forschung, wissenschaft, ja, mehr, ichbinhanna",352,männlich,0_forschung_ichbinhanna_wissenschaft_meh...
2,1,"lehre, forschung, prof, digitale, freuen",160,männlich,1_lehre_forschung_freuen_chatgpt
3,2,"berlin, deutschland, forschung, transfer, exzellente",173,männlich,2_berlin_deutschland_forschung_transfer
4,3,"frauen, käthe, gender, wissenschaft, mädchen",28,männlich,3_frauen_gender_genderstudies_käthe
5,4,"mecfs, versorgung, mecfsimbundestag, longcovid, med",55,männlich,4_mecfs_mecfsimbundestag_versorgung_long...
6,5,"kernfusion, alternative, cdu, akutheit, energiekrise",81,männlich,5_kernfusion_alternative_energie_cdu


In [55]:
topics_per_class[31:37]

,Topic,Words,Frequency,Class,Name
31,0,"forschung, ichbinhanna, wissenschaft, lehre, mehr",215,weiblich,0_forschung_ichbinhanna_wissenschaft_meh...
32,1,"lehre, forschung, freuen, chatgpt, the",122,weiblich,1_lehre_forschung_freuen_chatgpt
33,2,"deutschland, berlin, transfer, forschung, job",54,weiblich,2_berlin_deutschland_forschung_transfer
34,3,"frauen, gender, genderstudies, personen, mädchen",91,weiblich,3_frauen_gender_genderstudies_käthe
35,4,"mecfs, longcovid, mecfsimbundestag, versorgung, jahrzehnten",57,weiblich,4_mecfs_mecfsimbundestag_versorgung_long...
36,5,"energie, getäuscht, kernwaffenteststoppvertrag, getarnt, saubere",19,weiblich,5_kernfusion_alternative_energie_cdu


In [59]:
  for i in range(1, 7):
    representation = topics_per_class.loc[i, 'Name']
    representative_words = topics_per_class.loc[i, 'Words']
    print(f"Thema {i}:")
    print(f"Name: {representation}")
    print(f"Representative Words: {representative_words}")
    print()

Thema 1:
Name: 0_forschung_ichbinhanna_wissenschaft_meh...
Representative Words: forschung, wissenschaft, ja, mehr, ichbinhanna

Thema 2:
Name: 1_lehre_forschung_freuen_chatgpt
Representative Words: lehre, forschung, prof, digitale, freuen

Thema 3:
Name: 2_berlin_deutschland_forschung_transfer
Representative Words: berlin, deutschland, forschung, transfer, exzellente

Thema 4:
Name: 3_frauen_gender_genderstudies_käthe
Representative Words: frauen, käthe, gender, wissenschaft, mädchen

Thema 5:
Name: 4_mecfs_mecfsimbundestag_versorgung_long...
Representative Words: mecfs, versorgung, mecfsimbundestag, longcovid, med

Thema 6:
Name: 5_kernfusion_alternative_energie_cdu
Representative Words: kernfusion, alternative, cdu, akutheit, energiekrise



In [61]:
  for i in range(31, 37):
    representation = topics_per_class.loc[i, 'Name']
    representative_words = topics_per_class.loc[i, 'Words']
    print(f"Thema {i}:")
    print(f"Name: {representation}")
    print(f"Representative Words: {representative_words}")
    print()

Thema 31:
Name: 0_forschung_ichbinhanna_wissenschaft_meh...
Representative Words: forschung, ichbinhanna, wissenschaft, lehre, mehr

Thema 32:
Name: 1_lehre_forschung_freuen_chatgpt
Representative Words: lehre, forschung, freuen, chatgpt, the

Thema 33:
Name: 2_berlin_deutschland_forschung_transfer
Representative Words: deutschland, berlin, transfer, forschung, job

Thema 34:
Name: 3_frauen_gender_genderstudies_käthe
Representative Words: frauen, gender, genderstudies, personen, mädchen

Thema 35:
Name: 4_mecfs_mecfsimbundestag_versorgung_long...
Representative Words: mecfs, longcovid, mecfsimbundestag, versorgung, jahrzehnten

Thema 36:
Name: 5_kernfusion_alternative_energie_cdu
Representative Words: energie, getäuscht, kernwaffenteststoppvertrag, getarnt, saubere



In [64]:
model.visualize_topics_per_class(topics_per_class, top_n_topics=60, normalize_frequency = True, height= 500)

#### Themen nach Wissenschafler*in - wissenschaftsnahe Person

In [65]:
topics_per_class3 = model.topics_per_class(docs,
    classes=df.author_category)

4it [00:00, 17.25it/s]


In [66]:
model.visualize_topics_per_class(topics_per_class3,
    top_n_topics=10, normalize_frequency = True)